# Calendar

Upcoming Research Coding Community workshops and events are organised below.


Events are read from `data/events.csv`. Ensure the file has the columns `title`, `club`, `start`, `end`, `location`, and `description` with date-times in ISO format such as `2025-10-07 13:00`. Use the controls to select a month and filter by club. Only events scheduled from today onward are shown.


In [ ]:
import html
from collections import defaultdict
from datetime import datetime, timedelta
from pathlib import Path
from uuid import uuid4

import pandas as pd
from IPython.display import HTML, Markdown, display

DATA_PATH = Path('data') / 'events.csv'
REQUIRED_COLUMNS = ['title', 'club', 'start', 'end', 'location', 'description']
PALETTE = ['#1976d2', '#2e7d32', '#8e24aa', '#ef6c00', '#00838f', '#5d4037']

def escape_text(value: str) -> str:
    return html.escape(value or '', quote=False)

def escape_attr(value: str) -> str:
    return html.escape(value or '', quote=True)

def load_events(path: Path) -> pd.DataFrame:
    if not path.exists():
        display(Markdown(f"Warning: `{path}` was not found. Add the file to populate the calendar."))
        return pd.DataFrame(columns=REQUIRED_COLUMNS)

    df = pd.read_csv(path)
    missing = [col for col in REQUIRED_COLUMNS if col not in df.columns]
    if missing:
        display(Markdown('Warning: events file is missing columns: ' + ', '.join(missing)))
        for col in missing:
            df[col] = ''

    df['start'] = pd.to_datetime(df['start'], errors='coerce')
    df['end'] = pd.to_datetime(df['end'], errors='coerce')
    df = df.dropna(subset=['start', 'end']).copy()
    df = df[df['end'] >= df['start']]

    for col in ['title', 'club', 'location', 'description']:
        df[col] = df[col].fillna('')
    df['title'] = df['title'].replace('', 'Untitled event')
    df['club'] = df['club'].replace('', 'Community')

    today = pd.Timestamp.today().normalize()
    df = df[df['end'] >= today].copy()
    if df.empty:
        display(Markdown('No upcoming events were found.'))
        return df

    df['month_key'] = df['start'].dt.to_period('M').dt.start_time
    df['start_time'] = df['start'].dt.strftime('%H:%M')
    df['end_time'] = df['end'].dt.strftime('%H:%M')
    df.sort_values(['start', 'end', 'title'], inplace=True)
    return df

def allocate_colors(df: pd.DataFrame) -> dict:
    club_map = {}
    for club in df['club'].unique():
        club_map[club] = PALETTE[len(club_map) % len(PALETTE)]
    return club_map

def render_month(month_key: pd.Timestamp, month_events: pd.DataFrame, colors: dict) -> tuple[str, str]:
    month_label = month_key.strftime('%B %Y')
    start_of_grid = month_key - timedelta(days=month_key.weekday())
    days = [start_of_grid + timedelta(days=offset) for offset in range(42)]

    events_by_day = defaultdict(list)
    for _, row in month_events.iterrows():
        events_by_day[row['start'].date()].append(row)

    today_date = datetime.today().date()
    day_cells = []

    for day in days:
        classes = ['calendar-day']
        if day.month != month_key.month:
            classes.append('outside')
        if day.date() == today_date:
            classes.append('today')

        cell_html = ["<div class='date-label'>{}</div>".format(day.day)]

        for row in events_by_day.get(day.date(), []):
            color = colors.get(row['club'], '#1976d2')
            location_html = ''
            if row['location']:
                location_html = "<span class='location'>{}</span>".format(escape_text(row['location']))
            description_html = ''
            if row['description']:
                description_html = "<span class='description'>{}</span>".format(escape_text(row['description']))

            event_html = (
                "<div class='calendar-event' data-club='{club}' style='background-color: {color};'>"
                "<span class='time'>{start}-{end}</span>"
                "<span class='title'>{title}</span>"
                "{location}{description}"
                "</div>"
            ).format(
                club=escape_attr(row['club']),
                color=escape_attr(color),
                start=escape_text(row['start_time']),
                end=escape_text(row['end_time']),
                title=escape_text(row['title']),
                location=location_html,
                description=description_html,
            )
            cell_html.append(event_html)

        day_cells.append("<div class='{}'>{}</div>".format(' '.join(classes), ''.join(cell_html)))

    month_html = "".join(day_cells)
    return month_label, month_html

def render_calendar(df: pd.DataFrame):
    if df.empty:
        return

    colors = allocate_colors(df)
    months = {month: group.copy() for month, group in df.groupby('month_key')}
    container_id = 'calendar-{}'.format(uuid4().hex)
    month_select_id = '{}-month'.format(container_id)
    club_select_id = '{}-club'.format(container_id)

    month_blocks = []
    month_options = []
    for month_key in sorted(months.keys()):
        label, month_html = render_month(month_key, months[month_key], colors)
        value = month_key.strftime('%Y-%m')
        month_options.append("<option value='{}'>{}</option>".format(value, html.escape(label)))
        month_blocks.append(
            "<div class='calendar-month' data-month='{}'>".format(value)
            + "<div class='calendar-grid'>{}</div>".format(month_html)
            + "</div>"
        )

    club_options = ["<option value='ALL'>All clubs</option>"]
    for club in sorted(df['club'].unique()):
        club_options.append("<option value='{}'>{}</option>".format(escape_attr(club), html.escape(club)))

    legend_items = []
    for club, color in colors.items():
        legend_items.append(
            "<span><span class='color-swatch' style='background-color: {}'></span>{}</span>".format(
                escape_attr(color), html.escape(club)
            )
        )

    style_block = """
<style>
.calendar-container { font-family: 'Inter', 'Helvetica', 'Arial', sans-serif; margin: 1rem 0; }
.calendar-controls { display: flex; align-items: center; gap: 0.75rem; margin-bottom: 0.75rem; flex-wrap: wrap; }
.calendar-controls label { font-weight: 600; }
.calendar-controls select { padding: 0.25rem 0.5rem; }
.calendar-month { display: none; }
.calendar-month.active { display: block; }
.calendar-grid { display: grid; grid-template-columns: repeat(7, minmax(140px, 1fr)); border: 1px solid #d0d7de; border-bottom: none; border-right: none; background-color: #fff; }
.calendar-day { border-right: 1px solid #d0d7de; border-bottom: 1px solid #d0d7de; min-height: 140px; padding: 6px; box-sizing: border-box; font-size: 0.8rem; }
.calendar-day:nth-child(7n) { border-right: none; }
.calendar-day.outside { background-color: #f6f8fa; color: #657786; }
.calendar-day.today { box-shadow: inset 0 0 0 2px #1976d2; }
.date-label { font-weight: 600; margin-bottom: 0.35rem; }
.calendar-event { color: #fff; border-radius: 6px; padding: 5px 6px; margin-bottom: 5px; line-height: 1.2; box-shadow: 0 1px 2px rgba(0, 0, 0, 0.15); }
.calendar-event .time { display: block; font-weight: 600; }
.calendar-event .title { display: block; }
.calendar-event .location { display: block; font-size: 0.72rem; opacity: 0.9; }
.calendar-event .description { display: block; font-size: 0.72rem; opacity: 0.9; margin-top: 2px; }
.calendar-legend { display: flex; flex-wrap: wrap; gap: 0.75rem; margin-top: 0.75rem; font-size: 0.85rem; align-items: center; }
.calendar-legend .color-swatch { width: 12px; height: 12px; border-radius: 50%; display: inline-block; margin-right: 0.35rem; }
@media (max-width: 900px) { .calendar-grid { grid-template-columns: repeat(2, minmax(160px, 1fr)); } }
</style>
"""

    script_block = """
<script>
(function() {{
    var container = document.getElementById('{container_id}');
    if (!container) return;
    var monthSelect = container.querySelector('#{month_select_id}');
    var clubSelect = container.querySelector('#{club_select_id}');
    var months = container.querySelectorAll('.calendar-month');

    function showMonth(value) {{
        months.forEach(function(block) {{
            if (block.dataset.month === value) {{
                block.classList.add('active');
            }} else {{
                block.classList.remove('active');
            }}
        }});
    }}

    function filterEvents() {{
        var selectedClub = clubSelect.value;
        var events = container.querySelectorAll('.calendar-event');
        events.forEach(function(evt) {{
            if (selectedClub === 'ALL' || evt.dataset.club === selectedClub) {{
                evt.style.display = '';
            }} else {{
                evt.style.display = 'none';
            }}
        }});
    }}

    monthSelect.addEventListener('change', function(evt) {{
        showMonth(evt.target.value);
        filterEvents();
    }});

    clubSelect.addEventListener('change', filterEvents);

    if (monthSelect.value) {{
        showMonth(monthSelect.value);
    }} else if (months.length) {{
        monthSelect.value = months[0].dataset.month;
        showMonth(monthSelect.value);
    }}

    filterEvents();
}})();
</script>
""".format(
        container_id=container_id,
        month_select_id=month_select_id,
        club_select_id=club_select_id,
    )

    controls_html = (
        "<div class='calendar-controls'>"
        + "<label for='{id}'>Month:</label>".format(id=month_select_id)
        + "<select id='{id}' aria-label='Select month'>{options}</select>".format(
            id=month_select_id,
            options=''.join(month_options),
        )
        + "<label for='{id}'>Club:</label>".format(id=club_select_id)
        + "<select id='{id}' aria-label='Filter by club'>{options}</select>".format(
            id=club_select_id,
            options=''.join(club_options),
        )
        + "</div>"
    )

    legend_html = ''
    if legend_items:
        legend_html = "<div class='calendar-legend'>{}</div>".format(''.join(legend_items))

    calendar_html = (
        style_block
        + "<div id='{id}' class='calendar-container'>".format(id=container_id)
        + controls_html
        + ''.join(month_blocks)
        + legend_html
        + "</div>"
        + script_block
    )

    display(HTML(calendar_html))

def show_calendar(path: Path):
    df = load_events(path)
    render_calendar(df)

show_calendar(DATA_PATH)
